In [1]:
!pip install pyomo

In [2]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import seaborn as sns


'''
Import data once finished 
'''

#comment

'\nImport data once finished \n'

In [10]:
def readDataFile(filename):
    
    Network = pd.read_excel(filename, sheet_name = 'Network', index_col = 0)
    PowerLineReadings = pd.read_excel(filename, sheet_name = 'PowerLineReadings', index_col = 0)
    Demand = pd.read_excel(filename, sheet_name = 'Demand', index_col = 0)
    PVGeneration = pd.read_excel(filename, sheet_name = 'PVGeneration', index_col = 0)
    Load = pd.read_excel(filename, sheet_name = 'Load', index_col = 0)
    PVFarm = pd.read_excel(filename, sheet_name = 'PVFarm', index_col = 0)
    ESS = pd.read_excel(filename, sheet_name = 'ESS', index_col = 0)
    Generators = pd.read_excel(filename, sheet_name = 'Generators', index_col = 0)
    #GeneratorStepSize = pd.read_excel(filename, sheet_name = 'GeneratorStepSize', index_col = 0)
    #GeneratorStepCost = pd.read_excel(filename, sheet_name = 'GeneratorStepCost', index_col = 0)
    
    return {"Network": Network, "PowerLineReadings": PowerLineReadings, "Demand": Demand, "PVGeneration": PVGeneration, 
        "Load": Load, "PVFarm": PVFarm, "ESS": ESS, "Generators": Generators}

In [17]:
def optimization(inData, modelType):
    
    #Grab Data
    Network = inData["Network"]
    PowerLineReadings = inData["PowerLineReadings"]
    Demand = inData["Demand"]
    PVGeneration = inData["PVGeneration"]
    Load = inData["Load"]
    PVFarm = inData["PVFarm"]
    ESS = inData["ESS"]
    Generators = inData["Generators"]
    #GeneratorStepSize = inData["GeneratorStepSize"]
    #GeneratorStepCost = inData["GeneratorStepCost"]
    
    
    #Create Model
    model = ConcreteModel()
    
    
    Buses = list(set(list(Network.From.unique()) + list(Network.To.unique())))

    
    #Define Sets   e.g set time steps etc.
    model.t = Set(ordered = True, initialize = Demand.index) 
    model.l = Set(ordered = True, initialize = Network.index)
    model.j = Set(ordered = True, initialize = Load.index)
    model.s = Set(ordered = True, initialize = ESS.index)
    model.i = Set(ordered = True, initialize = Generators.index)
    model.k = Set(ordered = True, initialize = PVFarm.index)
    model.n = Set(ordered = True, initialize = Buses)
    
    #Define Parameters (unsure what we will be using yet, start simple and expand) e.g. Pmin
    model.Pmax = Param(model.i, mutable = True, within = NonNegativeReals)
    model.Pmin = Param(model.i, mutable = True, within = NonNegativeReals)
    model.P = Var(model.i, model.t, within = NonNegativeReals)
    model.u = Var(model.i, model.t, within = Binary)
    model.Pini = Param(model.i, mutable = True, within = NonNegativeReals)
    model.uini = Param(model.i, mutable = True, within = Binary)
    model.RU = Param(model.i, mutable = True, within = NonNegativeReals)
    model.RD = Param(model.i, mutable = True, within = NonNegativeReals)
    model.CSD = Var(model.i, model.t, within = NonNegativeReals)
    model.CSU = Var(model.i, model.t, within = NonNegativeReals)
    model.SOE = Var(model.s, model.t, within = NonNegativeReals)
    model.SUC = Param(model.i, mutable = True, within = NonNegativeReals)
    model.SDC = Param(model.i, mutable = True, within = NonNegativeReals)
    model.SOEini = Param(model.s, mutable = True, within = NonNegativeReals)
    model.Pcharge = Var(model.s, model.t, within = NonNegativeReals)
    model.Pdischarge = Var(model.s, model.t, within = NonNegativeReals)
    model.uess = Var(model.s, model.t, within = Binary)
    model.Eff = Param(model.s, mutable = True, within = NonNegativeReals)
    model.SOEmax = Param(model.s, mutable = True, within = NonNegativeReals)
    model.PV = Var(model.k, model.t, within = NonNegativeReals)
    model.f = Var(model.l, model.t, within = Reals)
    model.D = Var(model.j, model.t, within = NonNegativeReals)
    model.X = Param(model.l, mutable = True, within = NonNegativeReals)
    model.Capacity = Param(model.l, mutable = True, within = NonNegativeReals)
    
    #Give Param values
    for i in model.i:
        model.Pmax[i] = Generators.loc[i, 'Pmax']
        model.Pmin[i] = Generators.loc[i, 'Pmin']
        model.Pini[i] = Generators.loc[i, 'Pini']
        model.uini[i] = Generators.loc[i, 'uini']
        model.RU[i] = Generators.loc[i, 'RU (W/min)']
        model.RD[i] = Generators.loc[i, 'RD (W/min)']
        model.SUC[i] = Generators.loc[i, 'SUC ']
        model.SDC[i] = Generators.loc[i, 'SDC']
        
    for s in model.s:
        model.SOEini[s] = ESS.loc[s, 'SOEini']
        model.Eff[s] = ESS.loc[s, 'Eff']
        
    for j in model.j:
        for t in model.t:
            model.D[j,t] = Demand.loc[t, 'Demand '] * Load.loc[j, 'Percentage']/100
    
    for k in model.k:
        for t in model.t:
            model.PV[k,t] = PVGeneration.loc[k, "PVGeneration"] * PVFarm.loc[t, 'Percentage']
            
    for l in model.l:
        model.X[l] = Network.loc[l, 'X']
        model.Capacity[l] = Network.loc[l, 'Capacity']
        
    #for l in model.l:
        
            
    #Define Constraints ////////////////////////////////////////////////////////////
    
    def PowerMin(model, i, t):
        model.PowerMin = model.P[i,t] >= model.Pmin[i] * model.u[i,t]
        return model.PowerMin

    def PowerMax(model, i, t):
        model.PowerMax = model.P[i,t] <= model.Pmax[i] * model.u[i,t]
        return model.PowerMax

    def RampUp(model, i, t):
        if model.t == 1:
            model.RampUpInitial = model.P[i,t] - model.Pini[i] <= 60*model.RU[i]
            return model.RampUpInitial
        if model.t > 1:
            model.RampUp = model.P[i,t] - model.P[i,t-1] <= 60*model.RU[i]
            return model.RampUp
    
    def RampDown(model, i, t):
        if model.t == 1:
            model.RDInitial = model.Pini[i] - model.P[i, t] <= 60*model.RD[i]
            return model.RDInitial
        if model.t > 1:
            model.RD = model.P[i,t-1] - model.P[i,t] <= 60*model.RD[i]
            return model.RD
    
    def CostStartUp(model, i, t):
        if model.t == 1:
            model.CSUInitial = model.CSU[i, t] >= model.SUC[i]*(model.u[i,t]-model.uini[i])
            return model.CSUInitial
        if model.t > 1:
            model.CSU = model.CSU[i,t] >= model.SUC[i]*(model.u[i,t]-model.u[i,t-1])
            return model.CSU

    def CostShutDown(model, i, t):
        if model.t == 1:
            model.CSDInitial = model.CSD[i,t] >= model.SDC[i]*(model.uini[i]- model.u[i,t])
            return model.CSDInitial
        if model.t > 1:
            model.CSD = model.CSD[i,t] >= model.SDC[i]*(model.u[i,t-1]-model.u[i,t])
            return model.CSD

    #Energy Storage System & Charging Station
    def StateOfEnergy(model, s, t):
        if model.t == 1:
            model.SOEInitial = model.SOE[s, t] == model.SOEini[s] + model.Eff[s] * model.Pcharge[s,t] - (model.Pdischarge[s,t]/model.Eff[s])
            return model.SOEInitial
        if model.t > 1:
            model.SOE = model.SOE[s, t] == model.SOE[s, t-1] + model.Eff[s] * model.Pcharge[s,t] - (model.Pdischarge[s,t]/model.Eff[s])
            return model.SOE

    def MaxStateOfEnergy(model, s, t):
        model.MaxSOE = model.SOE[s,t] <= model.SOEmax[s]
        return model.MaxSOE

    def Pcharge(model, s, t):
        model.Pcharging = model.Pcharge[s, t] <= model.Pmax[s]*model.uess[s,t]
        return model.Pcharging

    def Pdischarge(model, s, t):
        model.Pdischarging = model.Pdischarge[s, t] <= model.Pmax[s] * (1 - model.uess[s,t])
        return model.Pdischarging

    #System
    def System(model, k, s, i, l, j, t):
        model.System = sum(model.PV[k,t]) + sum(model.Pdischarge[s,t]) + sum(model.P[i,t]) + sum(model.f[l,t]) == sum(model.D[j,t]) + sum(model.Pcharge[s,t]) + sum(model.f[l,t])
        return model.System

    #Objective
    
    model.PowerMaxConstraint = Constraint(model.i, model.t, rule = PowerMax)
    model.PowerMinConstraint = Constraint(model.i, model.t, rule = PowerMin)
    
    model.CostFnct = Objective(rule = )
    
    if modelType == 'Case Simple':
        model.Test = Constraint(model.t, rule = System)

    return model


    #e.g. CostFunction, PowerMin, PowerMax, RampUp, ON/OFF(binary), Battery
    #Later consider Q?
    
    #///////////////////////////////////////////////////////////////////////////////
    
    #create several Cases (no rampup, transmission line, network)
    

    
    

In [13]:
#//////////////////////////////////////////////

# TEST CASES($)
filename = 'InputData (3).xlsx'
readData = readDataFile(filename)
#model = optimization(readData, 'Case Simple')

#opt = pyo.SolverFactory('glpk')
#results = opt.solve(model)

#print(model.Objective)

#opt-Solver('WhatWeUseForSolving')

#//////////////////////////////////////////////

In [ ]:
model = optimization(readData, 'Case Simple')


In [19]:
filename = 'InputData (3).xlsx'
readData = readDataFile(filename)
readData['Load']



,Location,Feeder Rating (kVA),Percentage
n,,,
4.0,Memorial Hall (South),750.0,8.000000
13.0,Evergreen Hall,300.0,3.200000
14.0,Mullica Hill,225.0,2.400000
21.0,"Chestnut Hall, Willow, Magnolia",500.0,5.333333
22.0,Edgewood Park Apartments,300.0,3.200000
23.0,Esbjornson Gymnasium,500.0,5.333333
24.0,Student Recreation Center,1000.0,10.666667
25.0,Mimosa Hall,300.0,3.200000
28.0,Winans Hall (Wellness Center),500.0,5.333333


In [ ]:
model = optimization(readData, 'Case Simple')